# Analisi

In [67]:
import geopandas as gpd

from my_paths import *

In [68]:
PATH_ANALISI_TEST_CLEAN = "../Data/Clean/Analisi/test.geojson"

In [80]:
gdf_ciclabili = gpd.read_file(PATH_CICLABILI_CLEAN).to_crs("EPSG:32632")
gdf_bikemi = gpd.read_file(PATH_BIKEMI_CLEAN).to_crs("EPSG:32632")
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN).to_crs("EPSG:32632")
gdf_biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN).to_crs("EPSG:32632")
gdf_impianti_sportivi = gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN).to_crs("EPSG:32632")
gdf_fontane = gpd.read_file(PATH_FONTANE_CLEAN).to_crs("EPSG:32632")
gdf_inquinamento = gpd.read_file(PATH_INQUINAMENTO_INGESTION_CLEAN).to_crs("EPSG:32632")

### Metodo per trovare intersezioni tra geometrie.

Per farlo si crea un "buffer" di tot distanza (esempio 1000m) attorno a tutti gli oggetti geometrici.
Se quando si usa "intersects.()" su di un altro oggetto geometrico quello va a intersecarsi con il buffer allora restituisce True altrimenti False. Si crea una Series di True e False.  
Qui usiamo il "not" (~) per capire quale parco NON si interseca con il buffer di 1km per ogni ciclabile e ci restituisce il geodataframe con i parchi più lontani da quella distanza.

In [81]:
buffer_union = gdf_ciclabili.buffer(1000).union_all()
parchi_non_vicini = gdf_parchi[~gdf_parchi.geometry.intersects(buffer_union)]

In [82]:
from shapely import LineString, MultiLineString
from shapely.ops import nearest_points

# Lista per raccogliere risultati
risultati = []

for idx_parco, parco in parchi_non_vicini.iterrows():
    
    # Calcolo distanza tra il parco e ogni tratto ciclabile
    distanze = gdf_ciclabili.geometry.distance(parco.geometry)
    
    # Trova il tratto ciclabile più vicino
    ciclabile_piu_vicina = gdf_ciclabili.loc[distanze.idxmin()]
    
    # Trova i due punti più vicini tra geometria del parco e del tratto
    punto_parco, punto_ciclabile = nearest_points(parco.geometry, ciclabile_piu_vicina.geometry)
    
    # Salva risultato
    risultati.append({
        'id_parco': parco.get('PARCO', idx_parco),  # personalizza la chiave ID se necessario
        'id_ciclabile': ciclabile_piu_vicina.get('id_amat', ciclabile_piu_vicina.name),
        'distanza_minima': punto_parco.distance(punto_ciclabile),
        'geometry': MultiLineString([LineString([punto_parco, punto_ciclabile])])
    })

# Crea GeoDataFrame del risultato
df_risultati = gpd.GeoDataFrame(risultati, geometry='geometry', crs="EPSG:32632").to_crs('EPSG:4326')

In [83]:
df_risultati

,id_parco,id_ciclabile,distanza_minima,geometry
0,PARCO DELLA CAVA DI MUGGIANO,276,1685.17654,"MULTILINESTRING ((9.06633 45.44065, 9.05828 45..."


In [84]:
print(df_risultati["geometry"][0])

MULTILINESTRING ((9.066330415685773 45.44065006707306, 9.05827527105213 45.45471916947457))
